<a href="https://colab.research.google.com/github/chobitsX/30-seconds-of-code-zh-CN/blob/master/nb/Qwen2.5_(1.5B)-Alpaca.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://docs.unsloth.ai/get-started/installing-+-updating).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save)


### News

**Read our [blog post](https://unsloth.ai/blog/r1-reasoning) for guidance on how to train reasoning models.**

Visit our docs for all our [model uploads](https://docs.unsloth.ai/get-started/all-our-models) and [notebooks](https://docs.unsloth.ai/get-started/unsloth-notebooks).


### Installation

In [3]:
%%capture
# Normally using pip install unsloth is enough

# Temporarily as of Jan 31st 2025, Colab has some issues with Pytorch
# Using pip install unsloth will take 3 minutes, whilst the below takes <1 minute:
!pip install --no-deps bitsandbytes accelerate xformers==0.0.29 peft trl triton
!pip install --no-deps cut_cross_entropy unsloth_zoo
!pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
!pip install --no-deps unsloth

### Unsloth

In [6]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    # Can select any from the below:
    # "unsloth/Qwen2.5-0.5B", "unsloth/Qwen2.5-1.5B", "unsloth/Qwen2.5-3B"
    # "unsloth/Qwen2.5-14B",  "unsloth/Qwen2.5-32B",  "unsloth/Qwen2.5-72B",
    # And also all Instruct versions and Math. Coding verisons!
    model_name = "unsloth/Qwen2.5-1.5B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.2.5: Fast Qwen2 patching. Transformers: 4.48.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/1.40G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/171 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/4.71k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/617 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [7]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.2.5 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


<a name="Data"></a>
### Data Prep
We now use the Alpaca dataset from [yahma](https://huggingface.co/datasets/yahma/alpaca-cleaned), which is a filtered version of 52K of the original [Alpaca dataset](https://crfm.stanford.edu/2023/03/13/alpaca.html). You can replace this code section with your own data prep.

**[NOTE]** To train only on completions (ignoring the user's input) read TRL's docs [here](https://huggingface.co/docs/trl/sft_trainer#train-on-completions-only).

**[NOTE]** Remember to add the **EOS_TOKEN** to the tokenized output!! Otherwise you'll get infinite generations!

If you want to use the `llama-3` template for ShareGPT datasets, try our conversational [notebook](https://colab.research.google.com/drive/1XamvWYinY6FOSX9GLvnqSjjsNflxdhNc?usp=sharing).

For text completions like novel writing, try this [notebook](https://colab.research.google.com/drive/1ef-tab5bhkvWmBOObepl1WgJvfvSzn5Q?usp=sharing).

In [8]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

# EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
EOS_TOKEN = ""
def formatting_prompts_func(examples):
    instructions = []
    inputs = []
    outputs = []
    texts = []

    # 处理每个对话
    for conversation in examples["conversations"]:
        if len(conversation) >= 3:
            instruction = conversation[0]["content"]
            input_text = conversation[1]["content"]
            output = conversation[2]["content"]

            # 使用Alpaca模板进行格式化
            text = alpaca_prompt.format(
                instruction,
                input_text,
                output
            ) + EOS_TOKEN
            instructions.append(instruction)
            inputs.append(input_text)
            outputs.append(output)
            texts.append(text)

    return {"instruction": instructions, "input": inputs, "output": outputs, "text": texts}
pass

from datasets import load_dataset
# dataset = load_dataset("yahma/alpaca-cleaned", split = "train")
dataset = load_dataset('json', data_files='data.jsonl', split='train')
dataset = dataset.map(formatting_prompts_func, batched = True,remove_columns=dataset.column_names)
print(111, dataset)
print(dataset[0])

111 Dataset({
    features: ['instruction', 'input', 'output', 'text'],
    num_rows: 68
})
{'instruction': '\'## 背景\\n\'\n\'你是一个智能点餐助手，需要分析用户的输入，判断用户当前的意图属于哪个场景。\\n\'\n\'请根据用户输入判断场景类型，并优化用户的查询语句。\\n\'\n\'## 场景优先级和判断原则\\n\'\n\'优先考虑与点餐直接相关的场景，按以下原则判断：\\n\'\n\'1. 任何与餐品相关的询问或需求，优先判断是否为明确的点餐意图\\n\'\n\'2. 即使上一个场景是GENERAL，只要当前输入与餐品相关，也要立即切换到相应的点餐场景\\n\'\n\'3. 只有在确定与点餐完全无关时，才使用GENERAL场景\\n\\n\'\n\'## 智能体场景类型\\n\'\n\'### 点餐流程相关（优先考虑）：\\n\'\n\'- COMBO_REC(组合套餐): 当用户表达明确的点餐意图或套餐需求，如：\\n\'\n\'  * 任何涉及"套餐"的需求："推荐个套餐"、"有什么套餐"、"一(数字)人套餐"\\n\'\n\'  * 直接点餐意图："给我来个xxx"、"我要点xxx"\\n\'\n\'  * 修改订单："可乐不要了,鸡翅改为3个"\\n\'\n\'  * 明确的具体餐品需求："来份薯条"、"要一个汉堡"\\n\'\n\'- DISH_REC(餐品推荐): 仅用于模糊的餐品咨询或信息查询，如：\\n\'\n\'  * 模糊的餐品咨询："有什么好吃的"、"有什么推荐"\\n\'\n\'  * 询问餐品信息："这个好吃吗"、"有什么小吃"\\n\'\n\'  * 无明确点餐意图的提问："有没有辣的"\\n\'\n\'- ORDER_CONFIRM_PAYMENT(下单并支付): 1. 用户表示选好或要下单，如"可以了"、"就这些"、"选好了"、"我要下单"; 2. 用户表示要支付，如"我要付款"、"怎么支付"\\n\'\n\'- ORDER_DETAIL(订单详情): 当用户询问订单相关信息，如：\\n\'\n\'  * 查询订单状态："我的订单怎么样了"、"订单到哪了"\\n\'\n\'  * 查看订单内容："我下单了哪

<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`!

In [9]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Map (num_proc=2):   0%|          | 0/68 [00:00<?, ? examples/s]

In [10]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
1.408 GB of memory reserved.


In [11]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 68 | Num Epochs = 8
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 18,464,768


Step,Training Loss
1,1.591200
2,1.588200
3,1.582600
4,1.530400
5,1.467100
6,1.395200
7,1.302000
8,1.206600
9,1.119700
10,1.020400


In [12]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

365.7209 seconds used for training.
6.1 minutes used for training.
Peak reserved memory = 2.459 GB.
Peak reserved memory for training = 1.051 GB.
Peak reserved memory % of max memory = 16.681 %.
Peak reserved memory for training % of max memory = 7.13 %.


<a name="Inference"></a>
### Inference
Let's run the model! You can change the instruction and input - leave the output blank!

**[NEW] Try 2x faster inference in a free Colab for Llama-3.1 8b Instruct [here](https://colab.research.google.com/drive/1T-YBVfnphoVc8E2E854qF3jdia2Ll2W2?usp=sharing)**

In [13]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "'## 背景\\n'\n'你是一个智能点餐助手，需要分析用户的输入，判断用户当前的意图属于哪个场景。\\n'\n'请根据用户输入判断场景类型，并优化用户的查询语句。\\n'\n'## 场景优先级和判断原则\\n'\n'优先考虑与点餐直接相关的场景，按以下原则判断：\\n'\n'1. 任何与餐品相关的询问或需求，优先判断是否为明确的点餐意图\\n'\n'2. 即使上一个场景是GENERAL，只要当前输入与餐品相关，也要立即切换到相应的点餐场景\\n'\n'3. 只有在确定与点餐完全无关时，才使用GENERAL场景\\n\\n'\n'## 智能体场景类型\\n'\n'### 点餐流程相关（优先考虑）：\\n'\n'- COMBO_REC(组合套餐): 当用户表达明确的点餐意图或套餐需求，如：\\n'\n'  * 任何涉及\"套餐\"的需求：\"推荐个套餐\"、\"有什么套餐\"、\"一(数字)人套餐\"\\n'\n'  * 直接点餐意图：\"给我来个xxx\"、\"我要点xxx\"\\n'\n'  * 修改订单：\"可乐不要了,鸡翅改为3个\"\\n'\n'  * 明确的具体餐品需求：\"来份薯条\"、\"要一个汉堡\"\\n'\n'- DISH_REC(餐品推荐): 仅用于模糊的餐品咨询或信息查询，如：\\n'\n'  * 模糊的餐品咨询：\"有什么好吃的\"、\"有什么推荐\"\\n'\n'  * 询问餐品信息：\"这个好吃吗\"、\"有什么小吃\"\\n'\n'  * 无明确点餐意图的提问：\"有没有辣的\"\\n'\n'- ORDER_CONFIRM_PAYMENT(下单并支付): 1. 用户表示选好或要下单，如\"可以了\"、\"就这些\"、\"选好了\"、\"我要下单\"; 2. 用户表示要支付，如\"我要付款\"、\"怎么支付\"\\n'\n'- ORDER_DETAIL(订单详情): 当用户询问订单相关信息，如：\\n'\n'  * 查询订单状态：\"我的订单怎么样了\"、\"订单到哪了\"\\n'\n'  * 查看订单内容：\"我下单了哪些餐品\"、\"订单详情\"\\n'\n'  * 查询历史订单：\"我之前点过什么\"、\"上次的订单\"\\n'\n'### 其他通用对话（最后考虑）：\\n'\n'- GENERAL(通用对话): 仅用于完全与点餐无关的对话，如：\\n'\n'  * 纯问候：\"你好\"、\"在吗\"\\n'\n'  * 营业信息：\"几点关门\"、\"地址在哪\"\\n'\n'  * 其他非餐品对话\\n'\n'## 点餐场景流程示例：\\n'\n'### 流程一：\\n'\n'1. 用户输入(如：\"给我来点喝的\"), 你判断用户想要推荐餐品，于是返回DISH_REC场景。\\n'\n'2. 用户继续输入(如: \"来杯可乐\"), 你判断用户想要组合套餐，于是返回COMBO_REC场景。\\n'\n'3. 用户输入需求(如: \"可乐不要了,鸡翅改为3个,再来个汉堡\"), 你判断用户想要更改套餐，于是返回COMBO_REC场景。\\n'\n'4. 用户输入需求(如：\"可以了\" 或 \"下单吧\"), 你判断用户选好了餐品，并且无需更改，于是返回ORDER_CONFIRM_PAYMENT场景。\\n'\n'5. 用户输入需求(如：\"怎么支付\" 或 \"我要付款\"), 你判断用户想要支付，于是返回ORDER_CONFIRM_PAYMENT场景。\\n'\n'### 流程二：\\n'\n'1. 用户输入(如：\"来一个5人餐\"), 你判断用户想要组合套餐，于是返回COMBO_REC场景。\\n'\n'2. 用户输入需求(如: \"可乐不要了,鸡翅改为3个,再来个汉堡\"), 你判断用户想要更改套餐，于是返回COMBO_REC场景。\\n'\n'3. 用户输入需求(如：\"可以了\" 或 \"下单吧\"), 你判断用户选好了餐品，并且无需更改，于是返回ORDER_CONFIRM_PAYMENT场景。\\n'\n'4. 用户输入需求(如：\"怎么支付\" 或 \"我要付款\"), 你判断用户想要支付，于是返回ORDER_CONFIRM_PAYMENT场景。\\n'\n'5. 用户输入需求(如：\"几点关门\" 或 \"地址在哪\"), 你判断用户想要了解营业信息，于是返回GENERAL场景。\\n'\n'6. 用户输入需求(如：\"你好\" 或 \"在吗\"), 你判断用户想要问候，于是返回GENERAL场景。\\n'\n\nYour response should respect the following requirements:\n1) 返回的结构化是否符合要求\n2) 响应数据的质量效果\n", # instruction
        "推荐一个套餐，1个大人2个小孩", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

['Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\n\'## 背景\\n\'\n\'你是一个智能点餐助手，需要分析用户的输入，判断用户当前的意图属于哪个场景。\\n\'\n\'请根据用户输入判断场景类型，并优化用户的查询语句。\\n\'\n\'## 场景优先级和判断原则\\n\'\n\'优先考虑与点餐直接相关的场景，按以下原则判断：\\n\'\n\'1. 任何与餐品相关的询问或需求，优先判断是否为明确的点餐意图\\n\'\n\'2. 即使上一个场景是GENERAL，只要当前输入与餐品相关，也要立即切换到相应的点餐场景\\n\'\n\'3. 只有在确定与点餐完全无关时，才使用GENERAL场景\\n\\n\'\n\'## 智能体场景类型\\n\'\n\'### 点餐流程相关（优先考虑）：\\n\'\n\'- COMBO_REC(组合套餐): 当用户表达明确的点餐意图或套餐需求，如：\\n\'\n\'  * 任何涉及"套餐"的需求："推荐个套餐"、"有什么套餐"、"一(数字)人套餐"\\n\'\n\'  * 直接点餐意图："给我来个xxx"、"我要点xxx"\\n\'\n\'  * 修改订单："可乐不要了,鸡翅改为3个"\\n\'\n\'  * 明确的具体餐品需求："来份薯条"、"要一个汉堡"\\n\'\n\'- DISH_REC(餐品推荐): 仅用于模糊的餐品咨询或信息查询，如：\\n\'\n\'  * 模糊的餐品咨询："有什么好吃的"、"有什么推荐"\\n\'\n\'  * 询问餐品信息："这个好吃吗"、"有什么小吃"\\n\'\n\'  * 无明确点餐意图的提问："有没有辣的"\\n\'\n\'- ORDER_CONFIRM_PAYMENT(下单并支付): 1. 用户表示选好或要下单，如"可以了"、"就这些"、"选好了"、"我要下单"; 2. 用户表示要支付，如"我要付款"、"怎么支付"\\n\'\n\'- ORDER_DETAIL(订单详情): 当用户询问订

 You can also use a `TextStreamer` for continuous inference - so you can see the generation token by token, instead of waiting the whole time!

In [14]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "'## 背景\\n'\n'你是一个智能点餐助手，需要分析用户的输入，判断用户当前的意图属于哪个场景。\\n'\n'请根据用户输入判断场景类型，并优化用户的查询语句。\\n'\n'## 场景优先级和判断原则\\n'\n'优先考虑与点餐直接相关的场景，按以下原则判断：\\n'\n'1. 任何与餐品相关的询问或需求，优先判断是否为明确的点餐意图\\n'\n'2. 即使上一个场景是GENERAL，只要当前输入与餐品相关，也要立即切换到相应的点餐场景\\n'\n'3. 只有在确定与点餐完全无关时，才使用GENERAL场景\\n\\n'\n'## 智能体场景类型\\n'\n'### 点餐流程相关（优先考虑）：\\n'\n'- COMBO_REC(组合套餐): 当用户表达明确的点餐意图或套餐需求，如：\\n'\n'  * 任何涉及\"套餐\"的需求：\"推荐个套餐\"、\"有什么套餐\"、\"一(数字)人套餐\"\\n'\n'  * 直接点餐意图：\"给我来个xxx\"、\"我要点xxx\"\\n'\n'  * 修改订单：\"可乐不要了,鸡翅改为3个\"\\n'\n'  * 明确的具体餐品需求：\"来份薯条\"、\"要一个汉堡\"\\n'\n'- DISH_REC(餐品推荐): 仅用于模糊的餐品咨询或信息查询，如：\\n'\n'  * 模糊的餐品咨询：\"有什么好吃的\"、\"有什么推荐\"\\n'\n'  * 询问餐品信息：\"这个好吃吗\"、\"有什么小吃\"\\n'\n'  * 无明确点餐意图的提问：\"有没有辣的\"\\n'\n'- ORDER_CONFIRM_PAYMENT(下单并支付): 1. 用户表示选好或要下单，如\"可以了\"、\"就这些\"、\"选好了\"、\"我要下单\"; 2. 用户表示要支付，如\"我要付款\"、\"怎么支付\"\\n'\n'- ORDER_DETAIL(订单详情): 当用户询问订单相关信息，如：\\n'\n'  * 查询订单状态：\"我的订单怎么样了\"、\"订单到哪了\"\\n'\n'  * 查看订单内容：\"我下单了哪些餐品\"、\"订单详情\"\\n'\n'  * 查询历史订单：\"我之前点过什么\"、\"上次的订单\"\\n'\n'### 其他通用对话（最后考虑）：\\n'\n'- GENERAL(通用对话): 仅用于完全与点餐无关的对话，如：\\n'\n'  * 纯问候：\"你好\"、\"在吗\"\\n'\n'  * 营业信息：\"几点关门\"、\"地址在哪\"\\n'\n'  * 其他非餐品对话\\n'\n'## 点餐场景流程示例：\\n'\n'### 流程一：\\n'\n'1. 用户输入(如：\"给我来点喝的\"), 你判断用户想要推荐餐品，于是返回DISH_REC场景。\\n'\n'2. 用户继续输入(如: \"来杯可乐\"), 你判断用户想要组合套餐，于是返回COMBO_REC场景。\\n'\n'3. 用户输入需求(如: \"可乐不要了,鸡翅改为3个,再来个汉堡\"), 你判断用户想要更改套餐，于是返回COMBO_REC场景。\\n'\n'4. 用户输入需求(如：\"可以了\" 或 \"下单吧\"), 你判断用户选好了餐品，并且无需更改，于是返回ORDER_CONFIRM_PAYMENT场景。\\n'\n'5. 用户输入需求(如：\"怎么支付\" 或 \"我要付款\"), 你判断用户想要支付，于是返回ORDER_CONFIRM_PAYMENT场景。\\n'\n'### 流程二：\\n'\n'1. 用户输入(如：\"来一个5人餐\"), 你判断用户想要组合套餐，于是返回COMBO_REC场景。\\n'\n'2. 用户输入需求(如: \"可乐不要了,鸡翅改为3个,再来个汉堡\"), 你判断用户想要更改套餐，于是返回COMBO_REC场景。\\n'\n'3. 用户输入需求(如：\"可以了\" 或 \"下单吧\"), 你判断用户选好了餐品，并且无需更改，于是返回ORDER_CONFIRM_PAYMENT场景。\\n'\n'4. 用户输入需求(如：\"怎么支付\" 或 \"我要付款\"), 你判断用户想要支付，于是返回ORDER_CONFIRM_PAYMENT场景。\\n'\n'5. 用户输入需求(如：\"几点关门\" 或 \"地址在哪\"), 你判断用户想要了解营业信息，于是返回GENERAL场景。\\n'\n'6. 用户输入需求(如：\"你好\" 或 \"在吗\"), 你判断用户想要问候，于是返回GENERAL场景。\\n'\n\nYour response should respect the following requirements:\n1) 返回的结构化是否符合要求\n2) 响应数据的质量效果\n", # instruction
        "来点吃的", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
'## 背景\n'
'你是一个智能点餐助手，需要分析用户的输入，判断用户当前的意图属于哪个场景。\n'
'请根据用户输入判断场景类型，并优化用户的查询语句。\n'
'## 场景优先级和判断原则\n'
'优先考虑与点餐直接相关的场景，按以下原则判断：\n'
'1. 任何与餐品相关的询问或需求，优先判断是否为明确的点餐意图\n'
'2. 即使上一个场景是GENERAL，只要当前输入与餐品相关，也要立即切换到相应的点餐场景\n'
'3. 只有在确定与点餐完全无关时，才使用GENERAL场景\n\n'
'## 智能体场景类型\n'
'### 点餐流程相关（优先考虑）：\n'
'- COMBO_REC(组合套餐): 当用户表达明确的点餐意图或套餐需求，如：\n'
'  * 任何涉及"套餐"的需求："推荐个套餐"、"有什么套餐"、"一(数字)人套餐"\n'
'  * 直接点餐意图："给我来个xxx"、"我要点xxx"\n'
'  * 修改订单："可乐不要了,鸡翅改为3个"\n'
'  * 明确的具体餐品需求："来份薯条"、"要一个汉堡"\n'
'- DISH_REC(餐品推荐): 仅用于模糊的餐品咨询或信息查询，如：\n'
'  * 模糊的餐品咨询："有什么好吃的"、"有什么推荐"\n'
'  * 询问餐品信息："这个好吃吗"、"有什么小吃"\n'
'  * 无明确点餐意图的提问："有没有辣的"\n'
'- ORDER_CONFIRM_PAYMENT(下单并支付): 1. 用户表示选好或要下单，如"可以了"、"就这些"、"选好了"、"我要下单"; 2. 用户表示要支付，如"我要付款"、"怎么支付"\n'
'- ORDER_DETAIL(订单详情): 当用户询问订单相关信息，如：\n'
'  * 查询订单状态："我的订单怎么样了"、"订单到哪了"\n'
'  * 查看订单内容："我下单了哪些餐品"、"订单详情"\n'
'  * 查询历

<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [15]:
model.save_pretrained("lora_model")  # Local saving
tokenizer.save_pretrained("lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/vocab.json',
 'lora_model/merges.txt',
 'lora_model/added_tokens.json',
 'lora_model/tokenizer.json')

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [16]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# alpaca_prompt = You MUST copy from above!

inputs = tokenizer(
[
    alpaca_prompt.format(
        "What is a famous tall tower in Paris?", # instruction
        "", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
What is a famous tall tower in Paris?

### Input:


### Response:
The Eiffel Tower is a famous tall tower in Paris.<|endoftext|>


You can also use Hugging Face's `AutoModelForPeftCausalLM`. Only use this if you do not have `unsloth` installed. It can be hopelessly slow, since `4bit` model downloading is not supported, and Unsloth's **inference is 2x faster**.

In [ ]:
if False:
    # I highly do NOT suggest - use Unsloth if possible
    from peft import AutoPeftModelForCausalLM
    from transformers import AutoTokenizer
    model = AutoPeftModelForCausalLM.from_pretrained(
        "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        load_in_4bit = load_in_4bit,
    )
    tokenizer = AutoTokenizer.from_pretrained("lora_model")

### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [ ]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

[**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/drive/1WZDi7APtQ9VsvOrQSSC5DDtxq159j8iZ?usp=sharing)

In [17]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if True: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        "hf/model", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "", # Get a token at https://huggingface.co/settings/tokens
    )

Unsloth: You have 1 CPUs. Using `safe_serialization` is 10x slower.
We shall switch to Pytorch saving, which might take 3 minutes and not 30 minutes.
To force `safe_serialization`, set it to `None` instead.
Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 1.4G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 6.79 out of 12.67 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


100%|██████████| 28/28 [00:00<00:00, 32.80it/s]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model/pytorch_model.bin...
Done.


Unsloth: Converting qwen2 model. Can use fast conversion = False.


==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits might take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['q4_k_m'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...
Unsloth: CMAKE detected. Finalizing some steps for installation.
Unsloth: [1] Converting model at model into f16 GGUF format.
The output location will be /content/model/unsloth.F16.gguf
This might take 3 minutes...
INFO:hf-to-gguf:Loading model: model
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:gguf: loading model part 'pytorch_model.bin'
INFO:hf-to-gguf:token_embd.weight,         torch.float16 --> F16, shape = {1536, 151936}
INFO:hf-to-gguf:blk.0.attn_q.bias,         torch.float16 --> F32, shape = {1536}
INFO:hf

In [18]:
!pwd

/content


In [19]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [20]:
import subprocess
subprocess.Popen(["ollama", "serve"])
import time
time.sleep(3) # Wait for a few seconds for Ollama to load!

In [22]:
print(tokenizer._ollama_modelfile)

AttributeError: Qwen2TokenizerFast has no attribute _ollama_modelfile

In [24]:
!pwd

/content


In [28]:
!ollama create unsloth_model2 -f ./model/Modelfile

gathering model components ⠙ gathering model components ⠹ gathering model components ⠸ gathering model components ⠼ gathering model components ⠴ gathering model components ⠦ gathering model components ⠧ gathering model components ⠧ gathering model components ⠏ gathering model components ⠋ gathering model components ⠙ gathering model components ⠙ gathering model components ⠸ gathering model components ⠼ gathering model components ⠴ gathering model components ⠦ gathering model components ⠧ gathering model components ⠇ gathering model components ⠇ gathering model components ⠋ gathering model components ⠙ gathering model components ⠹ gathering model components ⠸ gathering model components ⠼ gathering model components ⠴ gathering model components ⠦ gathering model components ⠧ gathering model components ⠇ gathering model components ⠏ gathering model components ⠋ gathering model components ⠙ gathering model components ⠙ gathering model components ⠸ gathering model components 
copying file sh

In [26]:
!curl http://localhost:11434/api/chat -d '{ \
    "model": "unsloth_model", \
    "messages": [ \
        { "role": "user", "content": "来点喝的" } \
    ] \
    }'

{"model":"unsloth_model","created_at":"2025-02-13T08:08:41.96910131Z","message":{"role":"assistant","content":"我"},"done":false}
{"model":"unsloth_model","created_at":"2025-02-13T08:08:42.026543023Z","message":{"role":"assistant","content":"建议"},"done":false}
{"model":"unsloth_model","created_at":"2025-02-13T08:08:42.049353646Z","message":{"role":"assistant","content":"你可以"},"done":false}
{"model":"unsloth_model","created_at":"2025-02-13T08:08:42.07748239Z","message":{"role":"assistant","content":"考虑"},"done":false}
{"model":"unsloth_model","created_at":"2025-02-13T08:08:42.089482945Z","message":{"role":"assistant","content":"以下"},"done":false}
{"model":"unsloth_model","created_at":"2025-02-13T08:08:42.107916616Z","message":{"role":"assistant","content":"几种"},"done":false}
{"model":"unsloth_model","created_at":"2025-02-13T08:08:42.125619827Z","message":{"role":"assistant","content":"饮品"},"done":false}
{"model":"unsloth_model","created_at":"2025-02-13T08:08:42.143441149Z","message":{"ro

Now, use the `model-unsloth.gguf` file or `model-unsloth-Q4_K_M.gguf` file in llama.cpp or a UI based system like Jan or Open WebUI. You can install Jan [here](https://github.com/janhq/jan) and Open WebUI [here](https://github.com/open-webui/open-webui)

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/unsloth) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Llama 3.2 Conversational notebook. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(1B_and_3B)-Conversational.ipynb)
2. Saving finetunes to Ollama. [Free notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)
3. Llama 3.2 Vision finetuning - Radiography use case. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(11B)-Vision.ipynb)
6. See notebooks for DPO, ORPO, Continued pretraining, conversational finetuning and more on our [documentation](https://docs.unsloth.ai/get-started/unsloth-notebooks)!

<div class="align-center">
  <a href="https://unsloth.ai"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a>

  Join Discord if you need help + ⭐️ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐️
</div>
